In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
listings = pd.read_csv('listings_tokyo.csv.gz')
# calender = pd.read_csv ('http://data.insideairbnb.com/japan/kantō/tokyo/2019-11-28/data/calendar.csv.gz')
# reviews = pd.read_csv('http://data.insideairbnb.com/japan/kantō/tokyo/2019-11-28/data/reviews.csv.gz')
# listings_vis = pd.read_csv ('http://data.insideairbnb.com/japan/kantō/tokyo/2019-11-28/visualisations/listings.csv')
# reviews_vis = pd.read_csv('http://data.insideairbnb.com/japan/kantō/tokyo/2019-11-28/visualisations/reviews.csv')
# neighborhood = pd.read_csv('http://data.insideairbnb.com/japan/kantō/tokyo/2019-11-28/visualisations/neighbourhoods.csv')
# neighborhood_geo = pd.read_csv('http://data.insideairbnb.com/japan/kantō/tokyo/2019-11-28/visualisations/neighbourhoods.geojson')


/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
listings.shape

(13762, 106)

In [3]:
cols_drop = ['id','listing_url','scrape_id','last_scraped','thumbnail_url','medium_url','picture_url','xl_picture_url',
             'host_id','host_url','host_name','host_thumbnail_url','host_picture_url','minimum_minimum_nights',
             'maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm',
             'maximum_nights_avg_ntm','calendar_last_scraped','host_location','name','space','description',
             'neighborhood_overview','notes','transit','access','interaction','house_rules','city','country','country_code',
             'state','host_about','host_neighbourhood','host_has_profile_pic','license','market','smart_location',
             'jurisdiction_names','square_feet','host_acceptance_rate','weekly_price','monthly_price','availability_60',
             'experiences_offered','is_business_travel_ready','has_availability','host_listings_count','host_total_listings_count',
             'requires_license','neighbourhood_group_cleansed','first_review','last_review','latitude','longitude',
             'street','neighbourhood','zipcode','bed_type','calendar_updated',
             'host_verifications','availability_365','calculated_host_listings_count', 
             'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 
             'calculated_host_listings_count_shared_rooms','number_of_reviews_ltm',
             'availability_30','availability_90', 'number_of_reviews', 'review_scores_rating',
             'review_scores_accuracy', 'review_scores_cleanliness',
             'review_scores_checkin', 'review_scores_communication',
             'review_scores_location', 'review_scores_value','reviews_per_month','host_response_rate']
listings_clean = listings.drop(columns=cols_drop,axis=1)

In [5]:
listings_clean_model = listings.drop(columns=cols_drop,axis=1)

In [6]:
# Drop any NaNs
listings_clean = listings_clean.dropna()
listings_clean.shape

(11995, 25)

In [ ]:
html = pandas_profiling.ProfileReport(listings_clean)
html 

In [7]:
# Apply this to features and convert them to numbers from Categorical 
def str_to_num(x):
    listings_clean[x] = listings_clean[x].str[1:-3]
    listings_clean[x] = listings_clean[x].str.replace(',','')
    listings_clean[x] = listings_clean[x].astype('int')   

In [8]:
str_to_num('cleaning_fee')
str_to_num('extra_people')
str_to_num('security_deposit')
str_to_num('price')

In [9]:
# Commenting since a first time host will not have any history of responses. 

# #bucketize response rate
# def bucketize():
#     listings_clean['host_response_rate'] = listings_clean['host_response_rate'].str[:-1].astype('int')
#     listings_clean['host_response_rate'] = pd.cut(listings_clean['host_response_rate'],
#                                                   bins=[0,50,80,90,95,99,100],
#                                                   labels=['0-49%','50-79%','80-89%','90-94%','95-99%','100%'],
#                                                   include_lowest=True)   

In [10]:
# bucketize()

In [11]:
# Calculate numbers of days host has been on AirBnB
# data was compiled on 28th Nov 2019
listings_clean['host_since'] = pd.to_datetime(listings_clean['host_since'])
listings_clean['host_active'] = (pd.datetime(2019, 11, 28)-
                                   listings_clean['host_since'])/np.timedelta64(1,'D')


In [12]:
# Clubbing Serviced apartment,Aparthotel, Loft,Condominium as apartment
# Clubbing Boutique hotel  and hotel
# Club Houuse, Guesthouse,Townhouse,Tiny house,Guest suite,Earth house,Cabin
# Club Villa and bungalow 

listings_clean['property_type'].replace({
    'Condominium':'Apartment',
    'Serviced apartment': 'Apartment',
    'Aparthotel':'Apartment',
    'Loft':'Apartment',
    'Boutique hotel':'Hotel',
    'Guesthouse':'House',
    'Townhouse':'House',
    'Tiny house':'House',
    'Guest suite':'House',
    'Earth house':'House',
    'Cabin':'House',
    'Bungalow':'Villa',
    'Ryokan (Japan)':'Other',
    'Hut':'Other',
    'Guest suite':'Other',
    'Bed and breakfast':'Other',
    'Camper/RV':'Other',
    'Dorm':'Other'
    },inplace = True)

In [13]:
# Replacing columns with f/t with 0/1
listings_clean.replace({'f': 0, 't': 1}, inplace=True)

In [14]:
# Function to retain important amenities 
def beautiful_amenities(df):
        # strip excess characters from column, convert each row to a list
        df['amenities'] = df['amenities'].str.replace('"','').str.replace('{','').str.replace('}','').str.split(',')
        # create new columns for 69 most commonly found amenities
        df['hot_tub'] = [True if 'Hot tub' in row else False for row in df['amenities']]
        df['wifi'] = [True if 'Wifi' or 'Internet' in row else False for row in df['amenities']]
        df['ac'] = [True if 'Air conditioning' in row else False for row in df['amenities']]
        df['kitchen'] = [True if 'Kitchen' in row else False for row in df['amenities']]
        df['washer'] = [True if 'Washer' in row else False for row in df['amenities']]
        df['tv'] = [True if 'TV' in row else False for row in df['amenities']]
        df['intercom'] = [True if 'Buzzer/wireless intercom' in row else False for row in df['amenities']]
        df['heating'] = [True if 'Heating' in row else False for row in df['amenities']]
        df['kid_friendly'] = [True if 'Family/kid friendly' in row else False for row in df['amenities']]
        df['dryer'] = [True if 'Dryer' in row else False for row in df['amenities']]
        df['smoke_detector'] = [True if 'Smoke detector' in row else False for row in df['amenities']]
        df['cm_detector'] = [True if 'Carbon monoxide detector' in row else False for row in df['amenities']]
        df['fire_extinguisher'] = [True if 'Fire extinguisher' in row else False for row in df['amenities']]
        df['essentials'] = [True if 'Essentials' in row else False for row in df['amenities']]
        df['shampoo'] = [True if 'Shampoo' in row else False for row in df['amenities']]
        df['hangers'] = [True if 'Hangers' in row else False for row in df['amenities']]
        df['hair_dryer'] = [True if 'Hair dryer' in row else False for row in df['amenities']]
        df['iron'] = [True if 'Iron' in row else False for row in df['amenities']]
        df['workspace'] = [True if 'Laptop friendly workspace' in row else False for row in df['amenities']]
        df['self_check_in'] = [True if 'Self check-in' in row else False for row in df['amenities']]
        df['lockbox'] = [True if 'Lockbox' in row else False for row in df['amenities']]
        df['dishware'] = [True if 'Dishes and silverware' in row else False for row in df['amenities']]
        df['safety_card'] = [True if 'Safety card' in row else False for row in df['amenities']]
        df['shades'] = [True if 'Room-darkening shades' in row else False for row in df['amenities']]
        df['hot_water'] = [True if 'Hot water' in row else False for row in df['amenities']]
        df['bed_linens'] = [True if 'Bed linens' in row else False for row in df['amenities']]
        df['microwave'] = [True if 'Microwave' in row else False for row in df['amenities']]
        df['coffee_maker'] = [True if 'Coffee maker' in row else False for row in df['amenities']]
        df['refrigerator'] = [True if 'Refrigerator' in row else False for row in df['amenities']]
        df['cooking_basics'] = [True if 'Cooking basics' in row else False for row in df['amenities']]
        df['stove'] = [True if 'Stove' in row else False for row in df['amenities']]
        df['patio_or_balcony'] = [True if 'Patio or balcony' in row else False for row in df['amenities']]
        df['long_term'] = [True if 'Long term stays allowed' in row else False for row in df['amenities']]
        df['no_stairs'] = [True if 'No stairs or steps to enter' in row else False for row in df['amenities']]
        df['first_aid'] = [True if 'First aid kit' in row else False for row in df['amenities']]
        df['fire_extinguisher'] = [True if 'Fire extinguisher' in row else False for row in df['amenities']]
        df['twenty_four_hour'] = [True if '24-hour check-in' in row else False for row in df['amenities']]
        df['private_entrance'] = [True if 'Private entrance' in row else False for row in df['amenities']]
        df['luggage_dropoff'] = [True if 'Luggage dropoff allowed' in row else False for row in df['amenities']]
        df['well_lit'] = [True if 'Well-lit path to entrance' in row else False for row in df['amenities']]
        df['other'] = [True if 'Other' in row else False for row in df['amenities']]
        df['paid_parking'] = [True if 'Paid parking off premises' in row else False for row in df['amenities']]
        df['pets_allowed'] = [True if 'Pets allowed' in row else False for row in df['amenities']]
        df['extra_pillow'] = [True if 'Extra pillows and blankets' in row else False for row in df['amenities']]
        df['ethernet'] = [True if 'Ethernet connection' in row else False for row in df['amenities']]
        df['wide_hallways'] = [True if 'Wide hallways' in row else False for row in df['amenities']]
        df['flat_path'] = [True if 'Flat path to guest entrance' in row else False for row in df['amenities']]
        df['wide_entrance'] = [True if 'Wide entrance' in row else False for row in df['amenities']]
        df['extra_space'] = [True if 'Extra space around bed' in row else False for row in df['amenities']]
        df['wide_clearance'] = [True if 'Wide clearance to shower' in row else False for row in df['amenities']]
        df['extra_space'] = [True if 'Extra space around bed' in row else False for row in df['amenities']]
        df['shower_head'] = [True if 'Handheld shower head' in row else False for row in df['amenities']]
        df['bedroom_lock'] = [True if 'Lock on bedroom door' in row else False for row in df['amenities']]
        df['greet'] = [True if 'Host greets you' in row else False for row in df['amenities']]
        df['keypad'] = [True if 'Key pad' in row else False for row in df['amenities']]
        df['crib'] = [True if "Pack 'n Play/travel crib" in row else False for row in df['amenities']]
        df['wheelchair'] = [True if 'Wheelchair accessible' in row else False for row in df['amenities']]
        df['cleaning'] = [True if 'Cleaning before checkout' in row else False for row in df['amenities']]
        df['kitchenette'] = [True if 'Kitchenette' in row else False for row in df['amenities']]
        df['full_kitchen'] = [True if 'Full kitchen' in row else False for row in df['amenities']]
        df['pocket_wifi'] = [True if 'Pocket wifi' in row else False for row in df['amenities']]
        df['events'] = [True if 'Suitable for events' in row else False for row in df['amenities']]
        df['smart_lock'] = [True if 'Smart lock' in row else False for row in df['amenities']]
        df['private_living'] = [True if 'Private living room' in row else False for row in df['amenities']]
        df['garden_backyard'] = [True if 'Garden or backyard' in row else False for row in df['amenities']]
        df['smoking'] = [True if 'Smoking allowed' in row else False for row in df['amenities']]
        df['oven'] = [True if 'Oven' in row else False for row in df['amenities']]
        df['single_level'] = [True if 'Single level home' in row else False for row in df['amenities']]
        df['crib'] = [True if 'Crib' in row else False for row in df['amenities']]
        return df

In [15]:
#Keep beautiful amenities as individual features and drop the rest
listings_clean = beautiful_amenities(listings_clean)

In [16]:
# Replace num values for bool
listings_clean.replace({False: 0, True: 1}, inplace=True)

In [17]:
#Drop ameneties and host_since after feature engineering 
col_drop2 = ['amenities','host_since']
listings_clean = listings_clean.drop(columns=col_drop2,axis=1)

In [18]:
listings_clean.head()

,summary,host_response_time,host_is_superhost,host_identity_verified,neighbourhood_cleansed,is_location_exact,property_type,room_type,accommodates,bathrooms,...,kitchenette,full_kitchen,pocket_wifi,events,smart_lock,private_living,garden_backyard,smoking,oven,single_level
3,"So close to busy centers, yet so peaceful! Jus...",within an hour,0,1,Setagaya Ku,1,Apartment,Entire home/apt,2,1.0,...,0,0,0,0,0,0,0,0,0,0
6,"Monthly private fully furnished house, cozy an...",within an hour,0,1,Setagaya Ku,1,House,Entire home/apt,3,1.0,...,0,0,0,0,0,0,0,0,1,0
7,..*+;.*:' Popular room in Airbnb@Tokyo *;+:..*...,within a few hours,1,1,Shinjuku Ku,1,Apartment,Entire home/apt,3,1.0,...,0,0,0,0,0,0,0,0,0,0
8,License M (Phone number hidden by Airbnb) Enti...,within an hour,1,1,Shibuya Ku,0,Apartment,Private room,6,1.0,...,0,0,1,0,0,0,0,0,0,0
9,Quiet room on top floor of our European-style ...,within an hour,1,1,Mitaka Shi,1,House,Private room,2,1.5,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
list(listings_clean.columns)